In [1]:
import numpy as np
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [2]:
env = GridworldEnv()

In [21]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy. S: 0, 1, 2, ..., 15. A: 0(UP), 1(RIGHT), 2(DOWN), 3(LEFT)
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    
    # Initiallize the value function for the next iteration update
    V_next = np.zeros(env.nS)
    
    num_updates = 0
    # value function after one iteration update
    for s in range(env.nS):
        for a in range(env.nA):
            (prob, next_state, reward, done) = env.P[s][a][0]
            
            # full backup
            V_next[s] += policy[s][a] * prob* (reward + discount_factor * V[next_state])
            
    changes = [abs(i - j) for i, j in zip(V_next,V)]
    num_updates += 1
    print("Reshaped Grid Value Function after {} updates:".format(num_updates))
    print(V_next.reshape(env.shape))
    print("")
    
    # stop policy evaluation once the value function change is less then theta for all states
    while any([change > theta for change in changes]):
        V = V_next
        V_next = np.zeros(env.nS)
        
        for s in range(env.nS):
            for a in range(env.nA):
                prob, next_state, reward, done = env.P[s][a][0]
                
                # full backup
                V_next[s] += policy[s][a] * prob * (reward + discount_factor * V[next_state])
                
        changes = [abs(i - j) for i, j in zip(V_next,V)]   
        
        num_updates += 1
        print("Reshaped Grid Value Function after {} updates:".format(num_updates))
        print(V_next.reshape(env.shape))
        print("")
            
    return np.array(V_next)

In [22]:
reward_list = []
for s in range(env.nS):
    prob, next_state, reward, done = env.P[s][0][0]
    reward_list.append(reward)
    
print('Rewards:')
print(np.array(reward_list).reshape(env.shape))

Rewards:
[[ 0. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1.  0.]]


In [23]:
# Evaluate a uniform policy
random_policy = np.ones([env.nS, env.nA]) / env.nA

v = policy_eval(random_policy, env)

Reshaped Grid Value Function after 1 updates:
[[ 0. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1.  0.]]

Reshaped Grid Value Function after 2 updates:
[[ 0.   -1.75 -2.   -2.  ]
 [-1.75 -2.   -2.   -2.  ]
 [-2.   -2.   -2.   -1.75]
 [-2.   -2.   -1.75  0.  ]]

Reshaped Grid Value Function after 3 updates:
[[ 0.     -2.4375 -2.9375 -3.    ]
 [-2.4375 -2.875  -3.     -2.9375]
 [-2.9375 -3.     -2.875  -2.4375]
 [-3.     -2.9375 -2.4375  0.    ]]

Reshaped Grid Value Function after 4 updates:
[[ 0.      -3.0625  -3.84375 -3.96875]
 [-3.0625  -3.71875 -3.90625 -3.84375]
 [-3.84375 -3.90625 -3.71875 -3.0625 ]
 [-3.96875 -3.84375 -3.0625   0.     ]]

Reshaped Grid Value Function after 5 updates:
[[ 0.        -3.65625   -4.6953125 -4.90625  ]
 [-3.65625   -4.484375  -4.78125   -4.6953125]
 [-4.6953125 -4.78125   -4.484375  -3.65625  ]
 [-4.90625   -4.6953125 -3.65625    0.       ]]

Reshaped Grid Value Function after 6 updates:
[[ 0.         -4.20898438 -5.50976562 -5.80078

 [-21.99529213 -19.99579301 -13.99716093   0.        ]]

Reshaped Grid Value Function after 156 updates:
[[  0.         -13.99731196 -19.99601681 -21.99554257]
 [-13.99731196 -17.99649104 -19.99604345 -19.99601681]
 [-19.99601681 -19.99604345 -17.99649104 -13.99731196]
 [-21.99554257 -19.99601681 -13.99731196   0.        ]]

Reshaped Grid Value Function after 157 updates:
[[  0.         -13.99745495 -19.9962287  -21.99577969]
 [-13.99745495 -17.99667771 -19.99625393 -19.9962287 ]
 [-19.9962287  -19.99625393 -17.99667771 -13.99745495]
 [-21.99577969 -19.9962287  -13.99745495   0.        ]]

Reshaped Grid Value Function after 158 updates:
[[  0.         -13.99759034 -19.99642932 -21.99600419]
 [-13.99759034 -17.99685444 -19.9964532  -19.99642932]
 [-19.99642932 -19.9964532  -17.99685444 -13.99759034]
 [-21.99600419 -19.99642932 -13.99759034   0.        ]]

Reshaped Grid Value Function after 159 updates:
[[  0.         -13.99771852 -19.99661926 -21.99621676]
 [-13.99771852 -17.99702177 -1

In [24]:
# Evaluate a policy that merely go down
merely_go_down_policy = np.vstack([np.array([0.3, 0.3, 0.1, 0.3])] * env.nS)

v_up = policy_eval(merely_go_down_policy, env)

Reshaped Grid Value Function after 1 updates:
[[ 0. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1.  0.]]

Reshaped Grid Value Function after 2 updates:
[[ 0.  -1.7 -2.  -2. ]
 [-1.7 -2.  -2.  -2. ]
 [-2.  -2.  -2.  -1.9]
 [-2.  -2.  -1.7  0. ]]

Reshaped Grid Value Function after 3 updates:
[[ 0.   -2.31 -2.91 -3.  ]
 [-2.31 -2.82 -3.   -2.99]
 [-2.91 -3.   -2.94 -2.77]
 [-3.   -2.91 -2.37  0.  ]]

Reshaped Grid Value Function after 4 updates:
[[ 0.    -2.848 -3.766 -3.972]
 [-2.83  -3.586 -3.91  -3.974]
 [-3.766 -3.892 -3.868 -3.61 ]
 [-3.946 -3.802 -2.992  0.   ]]

Reshaped Grid Value Function after 5 updates:
[[ 0.     -3.3428 -4.5668 -4.9104]
 [-3.3014 -4.2656 -4.7846 -4.9178]
 [-4.541  -4.7462 -4.7228 -4.4356]
 [-4.8488 -4.6292 -3.6002  0.    ]]

Reshaped Grid Value Function after 6 updates:
[[ 0.      -3.79944 -5.32446 -5.80806]
 [-3.7242  -4.90326 -5.59734 -5.8274 ]
 [-5.26146 -5.52174 -5.54994 -5.22286]
 [-5.69058 -5.42148 -4.16562  0.     ]]

Reshaped Grid Va

 [-18.81938803 -17.97526887 -13.61342245   0.        ]]

Reshaped Grid Value Function after 167 updates:
[[  0.         -11.63055952 -18.68944797 -22.04566604]
 [-10.33835821 -15.34613349 -19.79842057 -22.11549187]
 [-16.33056748 -18.16005842 -19.53200017 -19.27735347]
 [-18.81948203 -17.97535968 -13.61349165   0.        ]]

Reshaped Grid Value Function after 168 updates:
[[  0.         -11.6306156  -18.68954411 -22.0457832 ]
 [-10.33840426 -15.34620733 -19.79852202 -22.11560889]
 [-16.33064343 -18.16014631 -19.5320989  -19.27745366]
 [-18.81957096 -17.9754456  -13.61355712   0.        ]]

Reshaped Grid Value Function after 169 updates:
[[  0.         -11.63066865 -18.68963507 -22.04589404]
 [-10.33844782 -15.34627719 -19.79861799 -22.1157196 ]
 [-16.3307153  -18.16022946 -19.53219231 -19.27754843]
 [-18.81965509 -17.97552688 -13.61361906   0.        ]]

Reshaped Grid Value Function after 170 updates:
[[  0.         -11.63071884 -18.68972113 -22.04599891]
 [-10.33848903 -15.34634328 -1

In [25]:
# Test: Make sure the evaluated policy is what we expected
expected_v = np.array([0, -14, -20, -22, -14, -18, -20, -20, -20, -20, -18, -14, -22, -20, -14, 0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)


print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")

Value Function:
[  0.         -13.99989315 -19.99984167 -21.99982282 -13.99989315
 -17.99986052 -19.99984273 -19.99984167 -19.99984167 -19.99984273
 -17.99986052 -13.99989315 -21.99982282 -19.99984167 -13.99989315   0.        ]

Reshaped Grid Value Function:
[[  0.         -13.99989315 -19.99984167 -21.99982282]
 [-13.99989315 -17.99986052 -19.99984273 -19.99984167]
 [-19.99984167 -19.99984273 -17.99986052 -13.99989315]
 [-21.99982282 -19.99984167 -13.99989315   0.        ]]

